In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# District Summary - Calculate the total number of schools
Num_of_schools_in_district = school_data_complete['school_name'].nunique()

In [3]:
# District Summary - Calculate the total number of students in the district
Num_of_students_in_district = school_data['size'].sum()

In [4]:
# District Summary - Calculate the total budget
total_district_budget = school_data_complete['budget'].unique().sum()
# District Summary - Calculate the average math score
ave_district_math_score = school_data_complete['math_score'].mean()
# District Summary - Calculate the average reading score
ave_district_reading_score = school_data_complete['reading_score'].mean()
#District Summary - 
#Calculate the overall passing rate (overall average score)
#i.e. (avg. math score + avg. reading score)/2
overall_pass_rate = (ave_district_math_score + ave_district_reading_score)/2

In [5]:
#District Summary - 
#Calculate the percentage of students with a passing math score (70 or greater)
passing_math_percentage = ((school_data_complete.math_score >= 70).sum()/Num_of_students_in_district)*100.0
#Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_percentage = ((school_data_complete.reading_score >= 70).sum()/Num_of_students_in_district)*100.0

In [6]:
# District Summary - Create a dataframe to hold the above results
district_df = pd.DataFrame(
    {"Total_Schools": [Num_of_schools_in_district],
     "Total_Students": [Num_of_students_in_district],
     "Total_Budget": [total_district_budget],
     "Average_Math_Score": [round(ave_district_math_score, 2)],
     "Average_Reading_Score": [round(ave_district_reading_score, 2)],
     "%Passing_Reading": [round(passing_reading_percentage, 2)],
     "%Passing_Math": [round(passing_math_percentage, 2)],
     "%Overall_Pass_Rate": [round(overall_pass_rate, 2)]
     })

# Optional: give the displayed data cleaner formatting
district_df["Total_Budget"] = district_df["Total_Budget"].map("${:,.2f}".format)
district_df["Average_Math_Score"] = district_df["Average_Math_Score"].map("{:.2f}%".format)
district_df["Average_Reading_Score"] = district_df["Average_Reading_Score"].map("{:.2f}%".format)
district_df["%Overall_Pass_Rate"] = district_df["%Overall_Pass_Rate"].map("{:.2f}%".format)
district_df["%Passing_Math"] = district_df["%Passing_Math"].map("{:.2f}%".format)
district_df["%Passing_Reading"] = district_df["%Passing_Reading"].map("{:.2f}%".format)
district_df["Total_Students"] = district_df["Total_Students"].map("{:,}".format)
district_df

,Total_Schools,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,%Passing_Reading,%Passing_Math,%Overall_Pass_Rate
0,15,"39,170","$24,649,428.00",78.99%,81.88%,85.81%,74.98%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [7]:
#School Summary - School Name, Type, Total Students, Total School Budget
#school_data dataframe initialized in the beginning already has all this information
#School Summary - Per Student Budget
school_data['Per_Student_Budget'] = school_data['budget']/school_data['size']

In [8]:
#School Summary - Average Math & Reading Scores
stu_stats_df = school_data_complete.groupby(['school_name'])
s = stu_stats_df.mean()
student_stat_ave = s[['reading_score', 'math_score']]
student_stat_ave = student_stat_ave.rename(
    columns={"reading_score": "Average_Reading_Score", "math_score": "Average_Math_Score"})

In [9]:
#merge the School Summary that includes the Per_Student_Budget column with the reading and math averages
all_schools_info = pd.merge(student_stat_ave, school_data, how="left", on=["school_name", "school_name"])

In [10]:
#Get the total of students that have passing reading and math scores
good_reading_scores =school_data_complete.loc[school_data_complete['reading_score']>=70,:]
passing_reading = pd.DataFrame(good_reading_scores['school_name'].value_counts().reset_index())
passing_reading = passing_reading.rename(
    columns={"school_name": "Total_Passing_Reading",
            "index": "school_name"})

good_math_scores =school_data_complete.loc[school_data_complete['math_score']>=70,:]
passing_math = pd.DataFrame(good_math_scores['school_name'].value_counts().reset_index())
passing_math = passing_math.rename(
    columns={"school_name": "Total_Passing_Math",
            "index": "school_name"})

read_math_totals = pd.merge(passing_math, passing_reading, how="left", on=["school_name", "school_name"])

In [11]:
#Add to the school summary the totals passing reading and math 
school_info = pd.merge(all_schools_info, read_math_totals, how="left", on=["school_name", "school_name"])

#Need to rename column 'size' so calculations are correct
school_info = school_info.rename(columns={"size": "Total_Students", "school_name": "School", "type": "Type", "budget": "Total_Budget"})

In [12]:
#School Summary - % Passing Math and % Passing Reading
school_info['%Passing_Math'] = ((school_info.Total_Passing_Math)/school_info.Total_Students)*100
school_info['%Passing_Reading'] = ((school_info.Total_Passing_Reading)/school_info.Total_Students)*100
#School Summary - Overall Pass Rate #i.e. (avg. math score + avg. reading score)/2
school_info['%Pass_Rate'] = ((school_info['%Passing_Reading'] + school_info['%Passing_Math'])/2)

In [13]:
#School Summary - Create a dataframe to hold the above results and take in only the columns that I need
complete_school_info = school_info.loc[:, ['School', 'Type', 'Total_Students', 'Total_Budget', 
                                           'Per_Student_Budget', 'Average_Math_Score', 'Average_Reading_Score', 
                                           '%Passing_Math', '%Passing_Reading', '%Pass_Rate']]
#complete_school_info dataframe holds the preserved formating for the original data
#complete_school_report dataframe is purely reformatted for aestitics 
complete_school_info = complete_school_info.set_index('School')


complete_school_report = complete_school_info.copy()
complete_school_report["Total_Budget"] = complete_school_report["Total_Budget"].map("${:,.2f}".format)
complete_school_report["Per_Student_Budget"] = complete_school_report["Per_Student_Budget"].map("${:,.2f}".format)
complete_school_report["Total_Students"] = complete_school_report["Total_Students"].map("{:,}".format)
complete_school_report

,Type,Total_Students,Total_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
School,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [14]:
top_performing_schools = complete_school_info.sort_values('%Pass_Rate', ascending=False).head()
top_performing_schools["Total_Budget"] = top_performing_schools["Total_Budget"].map("${:,.2f}".format)
top_performing_schools["Per_Student_Budget"] = top_performing_schools["Per_Student_Budget"].map("${:,.2f}".format)
top_performing_schools["Total_Students"] = top_performing_schools["Total_Students"].map("{:,}".format)
top_performing_schools

,Type,Total_Students,Total_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
School,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [15]:
bottom_performing_schools = complete_school_info.sort_values('%Pass_Rate').head()
bottom_performing_schools["Total_Budget"] = bottom_performing_schools["Total_Budget"].map("${:,.2f}".format)
bottom_performing_schools["Per_Student_Budget"] = bottom_performing_schools["Per_Student_Budget"].map("${:,.2f}".format)
bottom_performing_schools["Total_Students"] = bottom_performing_schools["Total_Students"].map("{:,}".format)
bottom_performing_schools

,Type,Total_Students,Total_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
School,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math & Reading Scores by Grade

* Create a table that lists the average Math & Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [16]:
#Get only the math grades and average it out by grade for each school
Math_byGrade = school_data_complete.groupby(['school_name', 'grade'])[['math_score']].mean().add_suffix('_Avg').reset_index()
Math_byGrade = Math_byGrade.rename(
     columns={"school_name": "School",
             "grade": "Grade",
             "math_score": "Average_Math_Score"})

#Separte out all the averages by school and store it into separate dataframes by grade
math9 = Math_byGrade.loc[Math_byGrade["Grade"] == "9th", :]
math10 = Math_byGrade.loc[Math_byGrade["Grade"] == "10th", :]
math11 = Math_byGrade.loc[Math_byGrade["Grade"] == "11th", :]
math12 = Math_byGrade.loc[Math_byGrade["Grade"] == "12th", :]

#merge all the math grades together by school name so that each column represents the ave grade for that grade
merge9and10 = pd.merge(math9, math10, on="School", how="outer")
merge9and10 = merge9and10.rename(columns={"math_score_Avg_x": "9th", "math_score_Avg_y": "10th"})
merge11and12 = pd.merge(math11, math12, on="School", how="outer")
merge11and12 = merge11and12.rename(columns={"math_score_Avg_x": "11th", "math_score_Avg_y": "12th"})
math = pd.merge(merge9and10, merge11and12, on="School", how='outer')

#remove the columns you don't need
allmath = math.loc[:, ['School', '9th', '10th', '11th','12th']]
allmath_report = allmath.copy()
allmath_report = allmath_report.set_index('School')
allmath_report

,9th,10th,11th,12th
School,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [17]:
#Get only the reading grades and average it out by grade for each school
read_byGrade = school_data_complete.groupby(['school_name', 'grade'])[['reading_score']].mean().add_suffix('_Avg').reset_index()
read_byGrade = read_byGrade.rename(
     columns={"school_name": "School",
             "grade": "Grade",
             "reading_score": "Average_Reading_Score"})

#Separte out all the averages by school and store it into separate dataframes by grade
read9 = read_byGrade.loc[read_byGrade["Grade"] == "9th", :]
read10 = read_byGrade.loc[read_byGrade["Grade"] == "10th", :]
read11 = read_byGrade.loc[read_byGrade["Grade"] == "11th", :]
read12 = read_byGrade.loc[read_byGrade["Grade"] == "12th", :]

#merge all the reading grades together by school name so that each column represents the ave grade for that grade
merge9and10read = pd.merge(read9, read10, on="School", how="outer")
merge9and10read = merge9and10read.rename(columns={"reading_score_Avg_x": "9th", "reading_score_Avg_y": "10th"})
merge11and12read = pd.merge(read11, read12, on="School", how="outer")
merge11and12read = merge11and12read.rename(columns={"reading_score_Avg_x": "11th", "reading_score_Avg_y": "12th"})
read = pd.merge(merge9and10read, merge11and12read, on="School", how='outer')

#remove the columns you don't need
allread = read.loc[:, ['School', '9th', '10th', '11th','12th']]
allread_report = allread.copy()
allread_report = allread_report.set_index('School')
allread_report

,9th,10th,11th,12th
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

by_spend = complete_school_info.copy()
by_spend = by_spend.reset_index()
by_spend["Spending Ranges (Per Student)"] = pd.cut(by_spend["Per_Student_Budget"], spending_bins, labels=group_names)
df_spend = by_spend.groupby('Spending Ranges (Per Student)')

new_df_spend= df_spend[['School', 'Average_Math_Score', 'Average_Reading_Score',
          '%Passing_Math', '%Passing_Reading', '%Pass_Rate']].mean()
new_df_spend

,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [19]:
size_bins = [0, 1000, 2000, 5000]
group_size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

by_size = complete_school_info.copy()
by_size = by_size.reset_index()
by_size["School_Size"] = pd.cut(by_spend["Total_Students"], size_bins, labels=group_size_names)
df_size = by_size.groupby('School_Size')

new_df_size = df_size[['School', 'Average_Math_Score', 'Average_Reading_Score',
          '%Passing_Math', '%Passing_Reading', '%Pass_Rate']].mean()
new_df_size

,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
School_Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [20]:
by_type = complete_school_info.copy()
by_type = by_type.reset_index()

df_type = by_type.groupby('Type').mean()

needed_df_type = df_type[['Average_Math_Score', 'Average_Reading_Score',
                          '%Passing_Math', '%Passing_Reading', '%Pass_Rate']]
needed_df_type

,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


# *Reports*

### District Summary

In [21]:
district_df

,Total_Schools,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,%Passing_Reading,%Passing_Math,%Overall_Pass_Rate
0,15,"39,170","$24,649,428.00",78.99%,81.88%,85.81%,74.98%,80.43%


### School Summary

In [22]:
complete_school_report

,Type,Total_Students,Total_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
School,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


### Top Performing Schools (By Passing Rate)

In [23]:
top_performing_schools

,Type,Total_Students,Total_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
School,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


### Bottom Performing Schools (By Passing Rate)

In [24]:
bottom_performing_schools

,Type,Total_Students,Total_Budget,Per_Student_Budget,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
School,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


### Math Scores by Grade

In [25]:
allmath_report

,9th,10th,11th,12th
School,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


### Reading Scores by Grade

In [26]:
allread_report

,9th,10th,11th,12th
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


### Scores by School Spending

In [27]:
new_df_spend

,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


### Scores by School Size

In [28]:
new_df_size

,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
School_Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


### Scores by School Type

In [29]:
needed_df_type

,Average_Math_Score,Average_Reading_Score,%Passing_Math,%Passing_Reading,%Pass_Rate
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
